In [ ]:
# Lib imports (enhanced - include mixed precision optional)
# ensure imports include applications
import os
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, callbacks, optimizers, applications
import numpy as np


In [13]:
# DATASET DIRECTORY CONFIGURATION (updated to use your dataset)
train_dir = r"C:\Users\Ralph\Documents\GitHub\25-26\CS 3B\RALPH_MARTIN_CHUA\ANN\train"
test_dir  = r"C:\Users\Ralph\Documents\GitHub\25-26\CS 3B\RALPH_MARTIN_CHUA\ANN\test"


In [ ]:
# IMAGE PARAMETERS (use ResNet input size)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32


In [ ]:
# DATA PREPROCESSING (ResNet)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    preprocessing_function=applications.resnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(preprocessing_function=applications.resnet.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print("Class indices:", train_generator.class_indices)
print(f"Train samples: {train_generator.samples}, Val samples: {val_generator.samples}, Test samples: {test_generator.samples}")


Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Class indices: {'chihuahua': 0, 'muffin': 1}
Train samples: 3788, Val samples: 945, Test samples: 1184


In [ ]:
# Build ResNet50V2 transfer-learning model
weight_decay = 1e-4
base_model = applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), pooling='avg')
base_model.trainable = False

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = layers.Dense(256, kernel_regularizer=regularizers.l2(weight_decay), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 batch_normalization_4 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                             

In [ ]:
# TRAIN / SAVE (use your lastname in the filename)
MODEL_FILENAME = 'exercise_6_custom_resnet.h5'  # replace custom_lastname with your last name if desired

checkpoint_path = MODEL_FILENAME
callbacks_list = [
    callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
]

history = model.fit(
    train_generator,
    epochs=12,
    validation_data=val_generator,
    callbacks=callbacks_list,
    verbose=1
)

# ensure final save (ModelCheckpoint already saved best model)
model.save(MODEL_FILENAME)
print(f"Saved improved model to {MODEL_FILENAME}")

Epoch 1/12
 51/119 [===========>..................] - ETA: 1:10 - loss: 0.7740 - accuracy: 0.6225

In [ ]:
# Load best-saved model and evaluate (uses preprocess_input via generators)
from tensorflow.keras.models import load_model
model = load_model(MODEL_FILENAME)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_acc:.4f}")


37/37 [==============================] - 25s 682ms/step - loss: 0.4925 - accuracy: 0.8623
Test Accuracy: 0.8623


In [ ]:
# PREDICTION + WRITE RESULTS (use ResNet preprocessing)
from ast import If


MODEL_FILENAME = MODEL_FILENAME

def predict_image_local(img_path, model_path=MODEL_FILENAME):
    model = tf.keras.models.load_model(model_path)
    from tensorflow.keras.preprocessing import image
    import numpy as np
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(applications.resnet.preprocess_input(img_array), axis=0)
    pred = float(model.predict(img_array)[0,0])
    label = "muffin" if pred >= 0.5 else "chihuahua"
    print(f"{img_path} -> Prediction: {label} (confidence: {pred:.4f})")
    return label, pred

# Example prediction placeholders - update with actual images if desired
label1, conf1 = predict_image_local(r"test/chihuahua/img_0_5.jpg")
label2, conf2 = predict_image_local(r"test/muffin/img_0_32.jpg")

#If predictions executed, write results:
with open('training_results.txt', 'w') as f:
  f.write(f"Test Accuracy: {test_acc:.4f}\n")
  f.write(f"Run 1 - {label1} (confidence: {conf1:.4f})\n")
  f.write(f"Run 2 - {label2} (confidence: {conf2:.4f})\n")
  print("Predictions saved to training_results.txt")

1/1 [==============================] - 0s 72ms/step
test/chihuahua/img_0_5.jpg -> Prediction: chihuahua (confidence: 0.0184)
1/1 [==============================] - 0s 124ms/step
test/muffin/img_0_32.jpg -> Prediction: muffin (confidence: 0.9918)
Predictions saved to training_results.txt
